<a href="https://colab.research.google.com/github/amyyq2/CourseProject/blob/main/comments_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

     |████████████████████████████████| 954 kB 36.5 MB/s 
     |████████████████████████████████| 138 kB 51.1 MB/s 
     |████████████████████████████████| 356 kB 42.9 MB/s 
     |████████████████████████████████| 3.5 MB 39.9 MB/s 
     |████████████████████████████████| 55 kB 2.9 MB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://ppa.laun

In [ ]:
import time
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd   
from selenium import webdriver

data=[]
# driver = webdriver.Chrome('/path/to/chromedriver') 
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
# driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

with Chrome('chromedriver',chrome_options=chrome_options) as driver:
    wait = WebDriverWait(driver,15)
    driver.get("https://www.youtube.com/watch?v=0EVVKs6DQLo")

    for item in range(40): 
        wait.until(EC.visibility_of_element_located((By.TAG_NAME, "body"))).send_keys(Keys.END)
        # time.sleep(15)

    for comment in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#content"))):
        data.append(comment.text)

df = pd.DataFrame(data, columns=['comment'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: use options instead of chrome_options


In [ ]:
%load_ext google.colab.data_table
df.iloc[2: , :]

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


,comment
2,This woman respected the covid 19 rules and we...
3,perfect
4,Popular opinion: All Doja Cat songs are so add...
5,SZA voice is just always so infectious and doj...
6,It is IMPOSSIBLE for me to get sick of this so...
...,...
111,In Love With This Song ️️️️️
112,I know Doja is happy to start a new era and do...
113,"This is my workout song. I dance high to Doja,..."
114,"SHE HAS A BEAUTIFUL VOICE, LOVE IT.."


# Data Cleaning
- Filter out the emojis
- Filter out the newlines
- Filter out non-english comments
- Filter out any empty rows
- Filter out EXACT duplicate comments


# Sentiment Analysis
- Find a model 
- Assuming we will output a numerical value for each sentence

# Parallelization/Multi-Threading